In [4]:
import sys
import os

cwd = os.getcwd()
sys.path.append(cwd)

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import math

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from torch.utils.data import Dataset, DataLoader, default_collate

from .hadl import HADL

from darts.datasets import ETTh1Dataset

torch.manual_seed(2003)

ImportError: attempted relative import with no known parent package

# Time Series Dataset Generator

## Time Series Dataset and Loader class

In [10]:
class CustomDataset(Dataset):
    """
    CustomDataset is a PyTorch dataset class designed for handling time-series data, providing support for different types of forecasting tasks:
    Univariate, Multivariate, and Multivariate-to-Univariate. This class allows for easy integration with PyTorch's DataLoader to generate
    batches of time-series sequences and corresponding prediction targets.

    Key Features:
    1. **Univariate Forecasting (type="S")**:
      - The input (`data_x`) and the target (`data_y`) are the same, predicting a single time series based on past values of that same series.

    2. **Multivariate Forecasting (type="M")**:
      - The input and target consist of all columns (features) of the time series. The task is to predict multiple series simultaneously.

    3. **Multivariate-to-Univariate Forecasting (type="MS")**:
      - The input (`data_x`) consists of all columns except the last, and the target (`data_y`) is the last column. This is for scenarios where
        multiple time series (features) are used to predict a single target series (last column).

    The dataset class handles sequence generation by segmenting the input data into overlapping subsequences for training, validation, and testing.
    It supports customization of sequence length (number of past time steps) and prediction length (future time steps to forecast).

    Parameters:
    - `data`: Input time-series data in the form of a NumPy array or pandas DataFrame.
    - `seq_len`: Length of the historical time-series window used as input for each sample.
    - `pred_len`: Length of the future time-series window used as the target for each sample.
    - `kind`: Type of forecasting task: "S" for Univariate, "M" for Multivariate, "MS" for Multivariate-to-Univariate.
    - `overlap`: Optional. Step size to create overlapping sequences. Default is 1, but can be set to higher values to create larger overlaps.

    This dataset class works seamlessly with PyTorch's DataLoader to generate mini-batches of time-series sequences for model training, evaluation, and testing.

    Note: The data must have target as last column.
    """

    def __init__(self, data, seq_len, pred_len, kind, overlap=1):
        self.data = data
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.overlap = overlap

        match kind:
            case "S":  # Univariate
                self.data_x = self.data[:,-1].reshape(-1,1)  # Input data (last column)
                self.data_y = self.data[:,-1].reshape(-1,1)  # Target data (last column)

            case "M":  # Multivariate
                self.data_x = self.data  # Input data (all columns)
                self.data_y = self.data  # Target data (all columns)

            case "MS":  # Multivariate to Univariate
                self.data_x = self.data[:, :-1]  # Input data (all except last column)
                self.data_y = self.data[:, -1].reshape(-1,1)  # Target data (only last column)

            case _:
                self.data_x = self.data[:,-1].reshape(-1,1)  # Input data (last column)
                self.data_y = self.data[:,-1].reshape(-1,1)  # Target data (last column)

    def __len__(self):
        return len(self.data) - self.seq_len - self.pred_len + 1

    def __getitem__(self, idx):
      seq_start_idx = idx * self.overlap
      seq_end_idx = seq_start_idx + self.seq_len

      pred_start_idx = seq_end_idx
      pred_end_idx = pred_start_idx + self.pred_len

      # If the prediction end index goes out of bounds, make it equal to len(data) and other indexes to be modified accordinly.
      if pred_end_idx > len(self.data):
         pred_end_idx = len(self.data)
         pred_start_idx = pred_end_idx - self.pred_len

         seq_end_idx = pred_end_idx
         seq_start_idx = seq_end_idx - self.seq_len


      seq_x = self.data_x[seq_start_idx:seq_end_idx, :]
      seq_y = self.data_y[pred_start_idx:pred_end_idx, :]

      return seq_x, seq_y

In [11]:
def Loader(time_series, batch_size=32, seq_len=96, pred_len=6, kind="S", overlap=4, split=(0.7,0.1,0.2), sc=StandardScaler()):
  """
    Loader function to create DataLoader objects for training, validation, and testing of time-series data.

    Parameters:
    - time_series: NumPy array or pandas DataFrame containing the time-series data.
    - batch_size: Number of samples per batch of data. Default is 32.
    - seq_len: Length of the input sequence (number of time steps for past data). Default is 96.
    - pred_len: Length of the prediction sequence (number of time steps to forecast). Default is 6.
    - kind: Type of forecasting task. "S" for univariate, "M" for multivariate, "MS" for multivariate-to-univariate. Default is "S".
    - overlap: Step size for overlapping sequences (affects how sequences are created). Default is 4.
    - split: Tuple representing the ratio of the dataset to be split into training, validation, and testing sets. Default is (0.7, 0.1, 0.2).

    Returns:
    - train_loader: DataLoader object for the training dataset.
    - val_loader: DataLoader object for the validation dataset.
    - test_loader: DataLoader object for the testing dataset.

    This function standardizes the data, splits it into train, validation, and test sets, and creates DataLoader objects
    with the specified batch size, sequence length, prediction length, and overlap for each set.

    Note:
    Shuffle is set to "False", since it is time series.
    Drop_last is set to "True", since while create sequences, last batch may not be same size as others.
    Time Series's target must be last column.
    """

  ## Borders for Train=70%, Validation=10% and Test=20%
  borders =(int(time_series.shape[0] * split[0]), int(time_series.shape[0] * (split[0] + split[1])), int(time_series.shape[0]))

  train_data = time_series[:borders[0],:]
  val_data = time_series[borders[0]:borders[1],:]
  test_data = time_series[borders[1]:borders[2],:]

  ## Perform standardization of the dataset.
  train_data = sc.fit_transform(train_data)
  val_data = sc.transform(val_data)
  test_data = sc.transform(test_data)

  ##
  train_data = torch.from_numpy(train_data).float()
  val_data = torch.from_numpy(val_data).float()
  test_data = torch.from_numpy(test_data).float()

  ## Create train, val and test loader.
  train_dataset = CustomDataset(train_data, seq_len=seq_len, pred_len=pred_len, kind=kind, overlap=overlap)
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

  val_dataset = CustomDataset(val_data, seq_len=seq_len, pred_len=pred_len, kind=kind, overlap=overlap)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

  test_dataset = CustomDataset(test_data, seq_len=seq_len, pred_len=pred_len, kind=kind, overlap=overlap)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

  return train_loader, val_loader, test_loader

## Train and Test Function

In [12]:
def train(model, train_loader, val_loader, num_epochs=30, patience=5, lr=0.01):
  # Training Loop
  train_losses = []
  val_losses = []
  best_val_loss = float('inf')
  early_stopping_counter = 0
  num_epochs = num_epochs
  patience = patience
  lr = lr

  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=patience, factor=0.5)

  for epoch in range(num_epochs):
      model.train()
      epoch_train_loss = 0.0
      for seq_x, seq_y in train_loader:
          #print(seq_x.shape)
          optimizer.zero_grad()
          outputs = model(seq_x)
          loss = criterion(outputs, seq_y)
          loss.backward()
          optimizer.step()
          epoch_train_loss += loss.item()

      # Calculate Average Train Loss
      avg_train_loss = epoch_train_loss / len(train_loader)
      train_losses.append(avg_train_loss)

      # Validation
      model.eval()
      epoch_val_loss = 0.0
      with torch.no_grad():
          for seq_x, seq_y in val_loader:
              outputs = model(seq_x)
              loss = criterion(outputs, seq_y)
              epoch_val_loss += loss.item()

      # Calculate Average Validation Loss
      avg_val_loss = epoch_val_loss / len(val_loader)
      val_losses.append(avg_val_loss)

      # Learning Rate Scheduling
      scheduler.step(avg_val_loss)

      # Early Stopping
      if avg_val_loss < best_val_loss:
          best_val_loss = avg_val_loss
          early_stopping_counter = 0
      else:
          early_stopping_counter += 1
          if early_stopping_counter >= patience:
              print(f'Early stopping at epoch {epoch+1}')
              break

      if epoch % 3 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

  return model


In [14]:
def test(model, test_loader):

  def MAE(pred, true):
    return np.mean(np.abs(pred - true))


  def MSE(pred, true):
      return np.mean((pred - true) ** 2)


  def RMSE(pred, true):
      return np.sqrt(MSE(pred, true))


  # Evaluation on Test Set
  model.eval()
  test_predictions = []
  test_actuals = []

  with torch.no_grad():
      for seq_x, seq_y in test_loader:
          outputs = model(seq_x)
          test_predictions.append(outputs.numpy())
          test_actuals.append(seq_y.numpy())


  test_predictions = np.concatenate(test_predictions)
  test_actuals = np.concatenate(test_actuals)


  print(test_predictions.shape, test_actuals.shape)

  # Calculate Metrics
  mae = MAE(test_actuals, test_predictions)
  mse = MSE(test_actuals, test_predictions)
  rmse = RMSE(test_actuals, test_predictions)

  print(f'Test MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}')

# Run the Model

## Load Dataset.

In [ ]:
time_series = ETTh1Dataset().load().values()
print(time_series.shape)

## Create train-val-test datasets.

In [ ]:
seq_len = 336 #Input Sequence Length
pred_len = 90 # Prediction Length
overlap = 4 # overlapping between each input length to create batches.
kind = "S" # M=Multivariate, S=univariate
batch_size=32 #batch size
split= (0.7,0.1,0.2) # train,val,,test split ratio.

train_loader, val_loader, test_loader = Loader(time_series,
                                               batch_size=batch_size,
                                               seq_len=seq_len,
                                               pred_len=pred_len,
                                               kind=kind,
                                               overlap=overlap,
                                               split=split,
                                               sc=StandardScaler())

In [ ]:
## Default settings
model = HADL(seq_len=seq_len, pred_len=pred_len, features=7, rank=30, individual=False, enable_Haar=True, enable_DCT=True)
mod_model = train(model, train_loader, val_loader)
test(mod_model, test_loader)


In [ ]:
## Enable Individual
model = HADL(seq_len=seq_len, pred_len=pred_len, features=7, rank=30, individual=True, enable_Haar=True, enable_DCT=True)
mod_model = train(model, train_loader, val_loader)
test(mod_model, test_loader)

In [ ]:
## Disable Haar
model = HADL(seq_len=seq_len, pred_len=pred_len, features=7, rank=30, individual=False, enable_Haar=False, enable_DCT=True)
mod_model = train(model, train_loader, val_loader)
test(mod_model, test_loader)

In [ ]:
## Disable DCT
model = HADL(seq_len=seq_len, pred_len=pred_len, features=7, rank=30, individual=False, enable_Haar=True, enable_DCT=False)
mod_model = train(model, train_loader, val_loader)
test(mod_model, test_loader)

In [ ]:
## Disable Haar and DCT
## Disable DCT
model = HADL(seq_len=seq_len, pred_len=pred_len, features=7, rank=30, individual=False, enable_Haar=False, enable_DCT=False)
mod_model = train(model, train_loader, val_loader)
test(mod_model, test_loader)